In [199]:
from sklearn.utils.class_weight import compute_sample_weight, compute_class_weight
from sklearn.model_selection import StratifiedKFold, GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import os
import sys
module_path = os.path.abspath(os.path.join('fede'))
if module_path not in sys.path:
    sys.path.append(module_path)
from client import Client
from fedavg import Fedavg
from sklearn.metrics import accuracy_score, f1_score
from sklearn.utils import shuffle
from supported_modles import Supported_modles
import random
import utils

In [200]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [201]:
selected_model = Supported_modles.logistic_regression

In [202]:
clients = utils.set_data()

Creating node1 on 0.0.0.0:11111.
Creating node2 on 0.0.0.0:11111.
Creating node3 on 0.0.0.0:11111.
Creating node4 on 0.0.0.0:11111.
Creating node5 on 0.0.0.0:11111.


In [211]:
clients = utils.set_data()
f1_before = []
for client in clients:
    client.downsample_data(['sbytes','dbytes','sttl','dttl','spkts','dpkts'])
    client.split_data()
    #client.prep_data()
    print(client.y.mean())
    client.train_model(selected_model)
    f1_before.append(client.f1)

Creating node1 on 0.0.0.0:11111.
Creating node2 on 0.0.0.0:11111.
Creating node3 on 0.0.0.0:11111.
Creating node4 on 0.0.0.0:11111.
Creating node5 on 0.0.0.0:11111.
0.5492537313432836
0.5522388059701493
0.5458208955223881
0.5501492537313433
0.5510626921534554


In [204]:
f1_before

[0.7655110656261994,
 0.7634749524413443,
 0.7538813947569356,
 0.7693467336683417,
 0.773734001112966]

In [205]:
selected_model = Supported_modles.SGD_classifier

In [206]:
fedavg = Fedavg("global",0.05)
fedavg.init_global_model(selected_model, clients[0].model,6)

In [207]:
model = clients[0].model
clients = utils.set_data()
for client in clients:
    client.downsample_data(['sbytes','dbytes','sttl','dttl','spkts','dpkts'])
    client.split_data()
    client.init_empty_model(selected_model,model)

Creating node1 on 0.0.0.0:11111.
Creating node2 on 0.0.0.0:11111.
Creating node3 on 0.0.0.0:11111.
Creating node4 on 0.0.0.0:11111.
Creating node5 on 0.0.0.0:11111.


In [208]:
number_of_rounds = 20
batch_size = 0.33
epochs = 20
prep = StandardScaler() 

for _ in range(number_of_rounds):
    print(f'Starting new round!')

    applicable_clients = random.sample((clients), random.randint(1, 4))
    applicable_models = []
    applicable_name = []
    round_weights = []
    dataset_size = 0
    

    for client in applicable_clients:
        # print(f'Client name: {client.name}')

        X = client.x
        y = client.y

        X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=batch_size, stratify=y, random_state=random.randint(0,10))
       
        #X_train = prep.fit_transform(X_train)

        dataset_size += X_train.shape[0]
        sample_weights = compute_sample_weight('balanced', y=y_train)

        fedavg.load_global_model(client.model, selected_model)
        fedavg.train_local_agent(X_train, y_train, client.model, epochs, sample_weights, selected_model)
        round_weights.append(X_train.shape[0])
        applicable_models.append(client.model)


    round_weights = np.array(round_weights) / dataset_size # calculate weight based on actual dataset size
    # round_weights = weights
    fedavg.update_global_model(applicable_models, round_weights, selected_model)

Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!


In [209]:
f1_fedavg = []
for client in clients:
    #x = prep.transform(client.x_test)
    y_hat = fedavg.model.predict(client.x_test)
    f1_fedavg.append(f1_score(client.y_test,y_hat))
print(f1_fedavg)

[0.8213462704669496, 0.8175271573294276, 0.8274025659646574, 0.8163163163163163, 0.8213462171955639]


In [210]:
i = 0
for client in clients:
    print(f'Difference {f1_fedavg[i]-f1_before[i]}')
    i += 1

Difference 0.05583520484075022
Difference 0.054052204888083266
Difference 0.07352117120772184
Difference 0.04696958264797457
Difference 0.047612216082597913


In [198]:
selected_model = Supported_modles.gradient_boosting_classifier
client = utils.centralized_data()
client.downsample_data(['sbytes','dbytes','sttl','dttl','spkts','dpkts'])
client.split_data()
client.train_model(selected_model)
print(client.f1)

Creating node1 on 0.0.0.0:11111.
0.8913590604026845
